## 1. Eliminación de variables por correlación

**Objetivo:** Análisis de correlación entre variables


**Información de las características**
Este dataset contiene imágenes que pertenecen all dataset de EuroSat. Hay 10 folders:
* 0 AnnualCrop
* 1 Forest
* 2 HerbaceousVegatation
* 3 Highway
* 4 Industrial
* 5 Pasture
* 6 PermanentCrop
* 7 Residential
* 8 River
* 9 SeaLake


**Número de instancias:** 27000

# 1. Autenticación por Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U kaleido
!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

# 2. Importando módulos

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import copy
import os
import plotly.express as px
import plotly.io as pio

# 3. Lectura de archivo

In [4]:
results_path = r'/content/drive/Shareddrives/Data Science para Geociencias/4. Selección de caracteristicas/Resultados'
filename = 'EUROSAT_TRAIN_FEAT.csv'
train_path = os.path.join(results_path, filename)

In [5]:
train_df = pd.read_csv(train_path)
train_df.head()

,histogram_0,histogram_1,histogram_2,histogram_3,histogram_4,histogram_5,histogram_6,histogram_7,histogram_8,histogram_9,histogram_10,histogram_11,histogram_12,histogram_13,histogram_14,histogram_15,histogram_16,histogram_17,histogram_18,histogram_19,histogram_20,histogram_21,histogram_22,histogram_23,histogram_24,histogram_25,histogram_26,histogram_27,histogram_28,histogram_29,histogram_30,histogram_31,histogram_32,histogram_33,histogram_34,histogram_35,histogram_36,histogram_37,histogram_38,histogram_39,...,histogram_493,histogram_494,histogram_495,histogram_496,histogram_497,histogram_498,histogram_499,histogram_500,histogram_501,histogram_502,histogram_503,histogram_504,histogram_505,histogram_506,histogram_507,histogram_508,histogram_509,histogram_510,histogram_511,hal_0,hal_1,hal_2,hal_3,hal_4,hal_5,hal_6,hal_7,hal_8,hal_9,hal_10,hal_11,hal_12,hum_0,hum_1,hum_2,hum_3,hum_4,hum_5,hum_6,label
0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012692,8.819525,0.606407,11.202328,0.411772,112.869246,35.989785,4.587024,6.990196,0.002292,2.686005,-0.132405,0.783513,0.002973,6.369633e-10,1.516469e-13,7.302349e-13,9.142081e-26,8.576981e-18,-2.251498e-25,Forest
1,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016780,6.737624,0.523612,7.070943,0.395274,113.673990,21.546147,4.192914,6.515763,0.002552,2.556843,-0.087996,0.659967,0.002942,2.693041e-10,9.678997e-15,7.353245e-13,1.652979e-26,1.191183e-17,5.979175e-26,Forest
2,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025098,4.159946,0.587513,5.044265,0.489949,105.225032,16.017115,4.011338,5.961651,0.003509,2.247017,-0.124186,0.727920,0.003155,4.341931e-10,1.849652e-12,2.115307e-13,-1.071065e-25,3.151259e-19,7.768625e-26,Forest
3,0.0,0.0,0.0,0.000718,0.00395,0.001436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022486,50.834805,0.744700,99.422595,0.436835,117.787398,346.855576,4.579942,6.767477,0.001086,2.938769,-0.190203,0.866096,0.002811,4.023647e-09,7.724173e-12,3.490115e-12,-1.568084e-23,6.745461e-17,9.082255e-24,Forest
4,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007640,12.731279,0.589228,15.496675,0.334405,113.976952,49.255423,4.815528,7.530149,0.001799,2.969903,-0.106116,0.746747,0.002936,4.838209e-10,3.726097e-12,5.626374e-12,2.496585e-23,1.228961e-16,-6.352761e-24,Forest


# 4. Análisis de correlación entre atributos

Aquí creamos una lista con los nombres de las características

In [6]:
attr_names = train_df.columns.to_list()
attr_names.pop(-1)
print(attr_names)

['histogram_0', 'histogram_1', 'histogram_2', 'histogram_3', 'histogram_4', 'histogram_5', 'histogram_6', 'histogram_7', 'histogram_8', 'histogram_9', 'histogram_10', 'histogram_11', 'histogram_12', 'histogram_13', 'histogram_14', 'histogram_15', 'histogram_16', 'histogram_17', 'histogram_18', 'histogram_19', 'histogram_20', 'histogram_21', 'histogram_22', 'histogram_23', 'histogram_24', 'histogram_25', 'histogram_26', 'histogram_27', 'histogram_28', 'histogram_29', 'histogram_30', 'histogram_31', 'histogram_32', 'histogram_33', 'histogram_34', 'histogram_35', 'histogram_36', 'histogram_37', 'histogram_38', 'histogram_39', 'histogram_40', 'histogram_41', 'histogram_42', 'histogram_43', 'histogram_44', 'histogram_45', 'histogram_46', 'histogram_47', 'histogram_48', 'histogram_49', 'histogram_50', 'histogram_51', 'histogram_52', 'histogram_53', 'histogram_54', 'histogram_55', 'histogram_56', 'histogram_57', 'histogram_58', 'histogram_59', 'histogram_60', 'histogram_61', 'histogram_62', '

Cálculo del coeficiente de Pearson entre pares de variables

In [7]:
vars = copy.deepcopy(attr_names)
for i in range(len(attr_names)):
  var1 = train_df.iloc[:,i]
  for j in range(0,i-1):
      var2 = train_df.iloc[:,j]
      corr,_ = pearsonr(var1,var2)
      if corr > 0.85:
        print('correlacion de %s entre %s y %s'%(corr,attr_names[i],attr_names[j]))
        try:
          vars.remove(attr_names[j])
        except:
          pass

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning:

An input array is constant; the correlation coefficent is not defined.



correlacion de 0.9999999999999926 entre histogram_34 y histogram_17
correlacion de 0.9834239045609443 entre histogram_39 y histogram_37
correlacion de 0.9999999999999928 entre histogram_42 y histogram_35
correlacion de 0.9326887889728163 entre histogram_93 y histogram_85
correlacion de 0.9999103997700494 entre histogram_166 y histogram_158
correlacion de 0.9994917829248796 entre histogram_167 y histogram_158
correlacion de 0.9994917829248796 entre histogram_173 y histogram_158
correlacion de 0.9998289494803698 entre histogram_173 y histogram_166
correlacion de 0.9999999999999963 entre histogram_173 y histogram_167
correlacion de 0.9994917829248796 entre histogram_174 y histogram_158
correlacion de 0.9998289494803698 entre histogram_174 y histogram_166
correlacion de 0.9999999999999963 entre histogram_174 y histogram_167
correlacion de 0.9994729197434423 entre histogram_237 y histogram_158
correlacion de 0.9998100845061444 entre histogram_237 y histogram_166
correlacion de 0.99998113811

Visualización de variables con correlación > 0.9

In [8]:
fig = px.scatter(data_frame=train_df,
                    x='hal_10',
                    y='hal_7',
                    color='label',
                    color_continuous_scale=px.colors.diverging.Tealrose,
                    template='seaborn',
                    title='Correlación',
                    width=1000,
                    height=500)
# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=25))
fig.update_traces(marker=dict(size=2),
                  selector=dict(mode='markers'))
pio.show(fig)
fig.write_html("corr.html")
fig.write_image("corr.png")

¿Con cuántas características nos quedamos?

In [9]:
print('Quedan %s variables NO dummies'%len(vars))
vars.append('label')

Quedan 517 variables NO dummies


Creando el dataframe sólo con las características con coef de Pearson < 0.9

In [10]:
train_df = train_df[train_df.columns.intersection(vars)]

Guardando el dataframe

In [ ]:
train_df.to_csv(os.path.join(results_path,'EUROSAT_CORR.csv'))